# 1. IMPORTS

In [2]:
#data_propbankbr lives within datasets
import sys
sys.path.append('../datasets/')


import pandas as pd 
from data_propbankbr import propbankbr_parser, propbankbr_argument_stats, propbankbr_split

In [3]:
# This dataframe has all multiple arguments on columns ARG0 ~ARG6
df = propbankbr_parser()
df.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG0,ARG1,ARG2,ARG3,ARG4,ARG5,ARG6
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,*,*,*,*,NaN,NaN,NaN
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,(A0*,*,*,*,NaN,NaN,NaN
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,*,*,*,*,NaN,NaN,NaN
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,*),*,*,*,NaN,NaN,NaN
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,(V*),*,*,*,NaN,NaN,NaN


 ### 1. 1 Normalization
Convert each record into a machine learning example i.e one argument per row
* Filter        .: each ARG column
* Create        .: tmp row holding the number of the argument ( for further ordering )
* Concatenate   .: everybody afterwards by rows
* Sort          .: reorder by S and TMP


In [42]:
columns_features= df.columns[:12]
df_feature= df.loc[:,columns_features].copy()

dataframes=[]
num_records=0
for i in range(7):
    col= 'ARG{:}'.format(i)
    df_target= df[col].dropna().to_frame()
    df_arg = df_feature.join(df_target, how='right')
    df_arg = df_arg.rename(index=str, columns={col: 'ARG'})
    df_arg['TMP']=i
    
    num_records+=df_arg.shape[0]
    print('{:}-\tnew records: {:}\ttotal records: {:}'.format(i, df_arg.shape[0],num_records))
    dataframes.append(df_arg)

#DataFrame.sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
df_norm = pd.concat(dataframes,axis=0)
df_norm.sort_values(by=['S','TMP'],axis=0 ,kind='mergesort', inplace=True)    
df_norm.head()

0-	new records: 69760	total records: 69760
1-	new records: 42196	total records: 111956
2-	new records: 19346	total records: 131302
3-	new records: 7114	total records: 138416
4-	new records: 2357	total records: 140773
5-	new records: 717	total records: 141490
6-	new records: 240	total records: 141730


,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG,TMP
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,*,0
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,(A0*,0
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,*,0
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,*),0
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,(V*),0


In [45]:
df_norm.head(50)

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG,TMP
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,*,0
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,(A0*,0
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,*,0
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,*),0
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,(V*),0
5,6,1,1,1,um,um,ART,M|S,7,>N,(NP*,-,(A1*,0
6,7,1,1,1,dado,dado,N,M|S,5,ACC,*,-,*,0
7,8,1,1,1,supreendente,surpreendente,ADJ,M|S,7,N<,(ADJP*),-,*,0
8,9,1,1,1,:,-,PU,-,7,PU,*,-,*,0
9,10,1,2,2,recusando,recusar,V-GER,-,25,ADVL,(FCL(ICL(VP*),recusar,*,0
